<a href="https://colab.research.google.com/github/syamsulmuariff/Rekomendasi_sistem/blob/main/sistem_rekomendasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Content Based Filtering : Rekomendasi Modul Pembelajaran**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/syamsulmuariff/Rekomendasi_sistem/main/data_recomendation.csv")
df.head()

,Nama perusahaan,Alamat,Modul
0,Syamsul mu arif,Ds.sumbermulyo Tlogowungu Pati,Sistem Digital Bahasa Pemprograman Algoritma d...
1,PT Nurul fikri cipta inovasi,jl.situ indah No 116 RT.006 RW.010 Kel tugu ke...,Desan web git/github javascript laravel databa...
2,PT Orbit Ventura Indonesia,Veteran RI Building 15th Floor Unit Z15-002 Pl...,Project AI Metode Penelitian AI Pemrograman Py...
3,PT Impactbyte Teknologi Edukasi,Jl. Simprug Golf 8 No.6 RT.2 RW.8 Grogol Sel....,User Interface Designing dan Prototyping Resea...
4,PT Mitra Integrasi Informatika,APL Tower Lantai 37 Jl.Letjen S.Parman Kav. 28...,Introduction Programming Front-End Programming...


**1. Ikhtisar**

In [2]:
df.describe()

,Nama perusahaan,Alamat,Modul
count,11,11,11
unique,10,11,11
top,PT Mitra Integrasi Informatika,Ds.sumbermulyo Tlogowungu Pati,Sistem Digital Bahasa Pemprograman Algoritma d...
freq,2,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Nama perusahaan  11 non-null     object
 1   Alamat           11 non-null     object
 2   Modul            11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


**2. Tampilan Modul Pembelajaran (Sebelum Preprocessing)**

In [16]:
def print_description(index):
    example = df[df.index == index][['Modul', 'Nama perusahaan','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama perusahaan:', example[1])
        print('Alamat:', example[2])

In [17]:
print_description(1)

Desan web git/github javascript laravel database MYSQL PHP RESTAPI UI/UX Soft skill final project
Nama perusahaan: PT Nurul fikri cipta inovasi
Alamat: jl.situ indah No 116 RT.006 RW.010 Kel tugu kec.cimanggis kota depok jawa barat


In [21]:
print_description(3)

User Interface Designing dan Prototyping Research dan Usability Testing UI/UX Design Thingking Career Development
Nama perusahaan: PT Impactbyte Teknologi Edukasi
Alamat:  Jl. Simprug Golf 8 No.6 RT.2 RW.8 Grogol Sel. Kec. Kby. Lama Kota Jakarta Selatan Daerah Khusus Ibukota Jakarta 12220


In [20]:
print_description(5)

Microsoft 365 Fundamentals Microsoft Azure Fundamentals Capstone Projetct Design Thingking Masterclass Modern Project Management Microsoft Azure Administrator
Nama perusahaan: PT MariBelajar Indonesia Cerdas
Alamat: Jln Tunggul Ametung IVa no 8 Desa Ubung Kaja Kecamatan Denpasar utara Kota Denpasar


**3. Text Preprocessing**

In [24]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modulpembelajaran_clean'] = df['Modul'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
df.head()

,Nama perusahaan,Alamat,Modul,modulpembelajaran_clean
0,Syamsul mu arif,Ds.sumbermulyo Tlogowungu Pati,Sistem Digital Bahasa Pemprograman Algoritma d...,sistem digital bahasa pemprograman algoritma d...
1,PT Nurul fikri cipta inovasi,jl.situ indah No 116 RT.006 RW.010 Kel tugu ke...,Desan web git/github javascript laravel databa...,desan web git github javascript laravel databa...
2,PT Orbit Ventura Indonesia,Veteran RI Building 15th Floor Unit Z15-002 Pl...,Project AI Metode Penelitian AI Pemrograman Py...,project ai metode penelitian ai pemrograman py...
3,PT Impactbyte Teknologi Edukasi,Jl. Simprug Golf 8 No.6 RT.2 RW.8 Grogol Sel....,User Interface Designing dan Prototyping Resea...,user interface designing dan prototyping resea...
4,PT Mitra Integrasi Informatika,APL Tower Lantai 37 Jl.Letjen S.Parman Kav. 28...,Introduction Programming Front-End Programming...,introduction programming frontend programming ...


**4. Tampilan Modul Pembelajaran (Setelah Preprocessing)**

In [30]:
# Tampilan kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modulpembelajaran_clean', 'Nama perusahaan','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama perusahaan:', example[1])
        print('Alamat:', example[2])

In [31]:
print_description_clean(1)

desan web git github javascript laravel database mysql php restapi ui ux soft skill final project
Nama perusahaan: PT Nurul fikri cipta inovasi
Alamat: jl.situ indah No 116 RT.006 RW.010 Kel tugu kec.cimanggis kota depok jawa barat


In [32]:
print_description_clean(2)

project ai metode penelitian ai pemrograman python logika konsep ai etika profesi proyek akhir
Nama perusahaan: PT Orbit Ventura Indonesia
Alamat: Veteran RI Building 15th Floor Unit Z15-002 Plaza Semanggi Jl.Jend.Sudirman Kav. 50 Jakarta Indonesia - 12930


In [33]:
print_description_clean(5)

microsoft 365 fundamentals microsoft azure fundamentals capstone projetct design thingking masterclass modern project management microsoft azure administrator
Nama perusahaan: PT MariBelajar Indonesia Cerdas
Alamat: Jln Tunggul Ametung IVa no 8 Desa Ubung Kaja Kecamatan Denpasar utara Kota Denpasar


**5. TF-IDF & Cosine Similarity**

In [34]:
df.set_index('Nama perusahaan', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modulpembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.02097435, 0.        , 0.0180986 , 0.        ,
        0.        , 0.        , 0.02736114, 0.02029287, 0.        ,
        0.01099162],
       [0.02097435, 1.        , 0.01287591, 0.05313647, 0.        ,
        0.0112824 , 0.03244877, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.01287591, 1.        , 0.        , 0.        ,
        0.01111059, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.0180986 , 0.05313647, 0.        , 1.        , 0.        ,
        0.04000823, 0.        , 0.05164984, 0.05141001, 0.        ,
        0.04176928],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.0112824 , 0.01111059, 0.04000823, 0.        ,
        1.        , 0.        , 0.03822683, 0.        , 0.        ,
        0.02582499],
       [0.        , 0.03244877, 0.        , 0.        , 0.

In [35]:
# Set index utama di kolom 'mitra'
indices = pd.Series(df.index)
indices[:50]

0                          Syamsul mu arif
1             PT Nurul fikri cipta inovasi
2               PT Orbit Ventura Indonesia
3          PT Impactbyte Teknologi Edukasi
4           PT Mitra Integrasi Informatika
5          PT MariBelajar Indonesia Cerdas
6      PT Abbauf Mulia Konsultan Teknologi
7        PT Telkom Indonesia (Persero) Tbk
8             PT Semesta Integrasi Digital
9           PT Mitra Integrasi Informatika
10    PT IlmuKomputerCom Braindevs Sistema
Name: Nama perusahaan, dtype: object

**6. Modelling**

In [36]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama modul berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 3 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul

**7. Prediksi**

In [39]:
recommendations('Syamsul mu arif')

['PT Telkom Indonesia (Persero) Tbk',
 'PT Nurul fikri cipta inovasi',
 'PT Semesta Integrasi Digital']

In [40]:
recommendations('PT Semesta Integrasi Digital')

['PT IlmuKomputerCom Braindevs Sistema',
 'PT Impactbyte Teknologi Edukasi',
 'PT Telkom Indonesia (Persero) Tbk']

In [41]:
recommendations('PT Abbauf Mulia Konsultan Teknologi')

['PT Nurul fikri cipta inovasi',
 'Syamsul mu arif',
 'PT Orbit Ventura Indonesia']